In [1]:
from qaravan.core import * 
from qaravan.tensorQ import *

In [22]:
print("Testing resets")

# Test 1: Reset should collapse superposition to computational basis state
print("Test 1: Reset collapses superposition")
gate_list = [H(0), Reset(0, "0")]
circ = Circuit(gate_list)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
expected = np.array([1, 0])
assert np.allclose(sv, expected), f"Expected |0⟩, got {sv}"
print("✓ Passed: Reset to |0⟩ from superposition")

gate_list = [H(0), Reset(0, "1")]
circ = Circuit(gate_list)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
expected = np.array([0, 1])
assert np.allclose(sv, expected), f"Expected |1⟩, got {sv}"
print("✓ Passed: Reset to |1⟩ from superposition")

# Test 2: Reset should preserve separable qubits
print("\nTest 2: Reset preserves other qubits")
gate_list = [H(0), X(1), Reset(0, "1")]
circ = Circuit(gate_list, n=2)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
expected = np.array([0, 0, 0, 1])  # |11⟩
assert np.allclose(sv, expected), f"Expected |11⟩, got {sv}"
print("✓ Passed: Other qubits preserved")

# Test 3: Reset should break entanglement
print("\nTest 3: Reset breaks entanglement")
gate_list = [H(0), CNOT([1,0]), Reset(0, "0")]
circ = Circuit(gate_list, n=2)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
# After reset to |0⟩, qubit 1 should be in superposition
# State should be (|00⟩ + |01⟩)/√2
expected = np.array([1, 0, 0, 0]) 
assert np.allclose(sv, expected), f"Expected (|00⟩+|01⟩)/√2, got {sv}"
print("✓ Passed: Entanglement broken correctly")

# Test 4: Multi-qubit reset
print("\nTest 4: Multi-qubit reset")
gate_list = [H(0), H(1), Reset([0,1], "10")]
circ = Circuit(gate_list, n=2)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
expected = np.array([0, 0, 1, 0])  # |10⟩
assert np.allclose(sv, expected), f"Expected |10⟩, got {sv}"
print("✓ Passed: Multi-qubit reset")

# Test 5: Sequential resets
print("\nTest 5: Sequential resets")
gate_list = [H(0), Reset(0, "1"), H(0), Reset(0, "1")]
circ = Circuit(gate_list)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
expected = -np.array([0, 1])
assert np.allclose(sv, expected), f"Expected |0⟩, got {sv}"
print("✓ Passed: Sequential resets")

# Test 6: Reset in the middle of a circuit
print("\nTest 6: Reset as ancilla qubit reset")
# Create Bell pair, reset ancilla, use it again
gate_list = [
    H(0), CNOT([1,0]),     # Create Bell pair
    Reset(1, "1"),          # Reset qubit 1 (ancilla)
    H(1),                   # Use ancilla again
    CNOT([2,1])             # Entangle with new qubit
]
circ = Circuit(gate_list, n=3)
sim = StatevectorSim(circ)
sim.run(progress_bar=False)
sv = sim.get_statevector()
# State: (|100⟩ - |111⟩) / √2
expected = (string_to_sv("100") - string_to_sv("111")) / np.sqrt(2)
assert np.allclose(sv, expected), f"Expected mixed state, got {sv}"
print("✓ Passed: Reset in middle of circuit")


Testing resets
Test 1: Reset collapses superposition
✓ Passed: Reset to |0⟩ from superposition
✓ Passed: Reset to |1⟩ from superposition

Test 2: Reset preserves other qubits
✓ Passed: Other qubits preserved

Test 3: Reset breaks entanglement
✓ Passed: Entanglement broken correctly

Test 4: Multi-qubit reset
✓ Passed: Multi-qubit reset

Test 5: Sequential resets
✓ Passed: Sequential resets

Test 6: Reset as ancilla qubit reset
✓ Passed: Reset in middle of circuit


In [18]:
print(sim)

1.0000+0.0000j|000⟩


In [2]:
gate_list = [H(0), CNOT([1,0]), Reset(0, "1")]
circ = Circuit(gate_list)
print(circ)

H gate on site(s) [0] 
bottom heavy CNOT2 gate on site(s) [0, 1] 
Reset on site(s) [0] to state |1⟩


In [7]:
sim = StatevectorSim(circ) 
sim.run(progress_bar=False)
print(sim)

1.0000+0.0000j|11⟩


In [6]:
sim = DensityMatrixSim(circ)
sim.run(progress_bar=False)
print(sim)

1.0000 * (1.0000|11⟩)


In [5]:
sim = MPSSim(circ)
sim.run(progress_bar=False)
print(sim)

1.0000|11⟩
